In [1]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime as dt
import time

In [4]:
data1 = pd.read_csv('../raw_data/cleaned_tweets_without_dupes_120323.csv', lineterminator='\n')

In [11]:
data1

,Unnamed: 0,datetime,username,text,date,negative_bert,neutral_bert,positive_bert
0,0,2021-01-01 22:09:59+00:00,sivilianlife,What's the chances @Grayscale is hedging short...,2021-01-01,0.002040,0.951312,0.046648
1,1,2021-01-01 22:09:39+00:00,LudicrousSpeed9,@CNBC Buy #Bitcoin to hedge against the result...,2021-01-01,0.004157,0.400750,0.595093
2,2,2021-01-01 22:09:35+00:00,SergioBowers1,@APompliano @pepsi @MonsterEnergy $DEO $KO e...,2021-01-01,0.000248,0.582107,0.417644
3,3,2021-01-01 22:09:21+00:00,bitvestor,Closed another chunk of this long now. Remaind...,2021-01-01,0.000974,0.807066,0.191960
4,4,2021-01-01 22:09:18+00:00,FROMKAIWITHLOVE,When you dig deeper into #kardiachain company ...,2021-01-01,0.000056,0.223449,0.776495
...,...,...,...,...,...,...,...,...
347663,347663,2023-03-09 06:00:01+00:00,HourlyBTCUpdate,Bitcoin: $21739.58\n💔 -1.72 last 1 Hour (-0.01...,2023-03-09,0.001016,0.870834,0.128149
347664,347664,2023-03-09 06:00:01+00:00,Mannny23,Popular crypto latest prices:\n $BTC 21751.00\...,2023-03-09,0.003519,0.907121,0.089360
347665,347665,2023-03-09 06:00:01+00:00,whalesradar_com,#CTXCUSDT #CTXC \nSignal #4 \n\nLast Signal: ...,2023-03-09,0.000521,0.964454,0.035024
347666,347666,2023-03-09 06:00:00+00:00,croxroadnews,The Wings of #Bitcoin: A Symbol of Strength an...,2023-03-09,0.007388,0.532228,0.460384


In [12]:
data1['datetime'] = pd.to_datetime(data1['datetime'])
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347668 entries, 0 to 347667
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   Unnamed: 0     347668 non-null  int64              
 1   datetime       347668 non-null  datetime64[ns, UTC]
 2   username       347668 non-null  object             
 3   text           347668 non-null  object             
 4   date           347668 non-null  object             
 5   negative_bert  347668 non-null  float64            
 6   neutral_bert   347668 non-null  float64            
 7   positive_bert  347668 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(3)
memory usage: 21.2+ MB


In [107]:
data1['datetime'].dt.date

0        2022-09-24
1        2022-09-24
2        2022-09-24
3        2022-09-24
4        2022-09-24
            ...    
39508    2022-12-23
39509    2022-12-23
39510    2022-12-23
39511    2022-12-23
39512    2022-12-23
Name: datetime, Length: 39513, dtype: object

In [15]:
def return_tweet_volume_scraped_data(data):
    start = time.time()
    i=1
    first_time = data1['datetime'][0]
    data1['tweet_per_10_mins'] = 0
    data1['segment_tweets'] = 0
    hour = data1['datetime'].dt.hour[0]
    for index, datetime in enumerate(data1['datetime']):
        if data1['datetime'].dt.hour[index] != hour:
            if i == 150:
                prev_ind = index-1
                data1['tweet_per_10_mins'][prev_ind] = 600*i/((first_time - last_time).total_seconds())
                i = 0
                first_time = data1['datetime'][index]
            else:
                prev_ind = index-1
                data1['tweet_per_10_mins'][prev_ind] = i
                i = 0
                first_time = data1['datetime'][index]
        else: 
            pass
        data1['segment_tweets'][index] = i
        i = i + 1
        hour = data1['datetime'].dt.hour[index]
        last_time = data1['datetime'][index]

    data2 = data1.groupby(by = 'date').sum()
    data2['tweets_per_day'] = data2['tweet_per_10_mins']*48
    end = time.time()
    print(end - start)
    return data2


In [16]:
return_tweet_volume_scraped_data(data1)

/tmp/ipykernel_8377/1523555119.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['segment_tweets'][index] = i
/tmp/ipykernel_8377/1523555119.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['tweet_per_10_mins'][prev_ind] = i
/tmp/ipykernel_8377/1523555119.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['tweet_per_10_mins'][prev_ind] = 600*i/((first_time - last_time).total_seconds())


KeyboardInterrupt: 

In [ ]:
sns.barplot(data=data2, x=data2.index, y='tweets_per_day' )